In [317]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [318]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_x.csv')
submission = pd.read_csv("submission_LGBM.csv")

In [319]:
submission_ids = test_df['index']

In [320]:
train_df.drop(columns=['index'], inplace=True)
test_df.drop(columns=['index'], inplace=True)
train_df

,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi
0,Kadın,31-40,Bekar,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,748266.440879,Büyük Şehir,Elektronik ve Teknolojik Ürünler,32010.900881,37.310644,Etmiyor,obek_4,102.936904
1,Erkek,>60,Evli,Lise Mezunu,Kendi İşinin Sahibi,246298.609605,Küçük Şehir,Spor Malzemeleri,4145.729891,11.554851,Etmiyor,obek_5,82.176053
2,Erkek,18-30,Bekar,Lise Mezunu,Düzenli ve Ücretli Bir İşi Var,268582.732360,Küçük Şehir,Giyim,9924.379414,55.353236,Ediyor,obek_8,141.661426
3,Erkek,51-60,Evli,Lise Mezunu,Kendi İşinin Sahibi,327721.837865,Kırsal,Giyim,6417.780714,15.417239,Etmiyor,obek_3,17.097397
4,Erkek,31-40,Evli,Ortaokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,397431.626948,Büyük Şehir,Giyim,7886.663061,14.689837,Etmiyor,obek_3,20.470459
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,Kadın,>60,Bekar,Üniversite Mezunu,Emekli,478054.247819,Köy veya Kasaba,Kozmetik,43182.299801,31.356284,Etmiyor,obek_6,51.600436
5456,Erkek,51-60,Evli,Doktora Ötesi,Kendi İşinin Sahibi,481852.853651,Köy veya Kasaba,Ev ve Mobilya,42107.996193,26.288620,Etmiyor,obek_6,49.190536
5457,Kadın,51-60,Evli,İlkokul Mezunu,Düzenli ve Ücretli Bir İşi Var,309443.687351,Küçük Şehir,Ev ve Mobilya,4655.392445,5.156906,Etmiyor,obek_1,9.537384
5458,Kadın,18-30,Bekar,Yüksekokul Mezunu,Düzenli ve Ücretli Bir İşi Var,266604.268016,Büyük Şehir,Ev ve Mobilya,9016.165906,39.205399,Ediyor,obek_8,143.281718


In [321]:
subDf = submission["Öbek İsmi"]
subDf = pd.DataFrame(subDf)
subDf

,Öbek İsmi
0,obek_3
1,obek_3
2,obek_2
3,obek_6
4,obek_1
...,...
2335,obek_8
2336,obek_2
2337,obek_7
2338,obek_5


In [322]:
merged_df = pd.concat([subDf, test_df], axis=1)

In [323]:
categorical_features = ['Cinsiyet', 'Yaş Grubu', 'Medeni Durum', 'Eğitim Düzeyi',
                        'İstihdam Durumu', 'Yaşadığı Şehir', 'En Çok İlgilendiği Ürün Grubu', 'Eğitime Devam Etme Durumu']

In [324]:
real_extended = pd.concat([train_df, merged_df], axis=0)

In [325]:
real_extended

,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi
0,Kadın,31-40,Bekar,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,748266.440879,Büyük Şehir,Elektronik ve Teknolojik Ürünler,32010.900881,37.310644,Etmiyor,obek_4,102.936904
1,Erkek,>60,Evli,Lise Mezunu,Kendi İşinin Sahibi,246298.609605,Küçük Şehir,Spor Malzemeleri,4145.729891,11.554851,Etmiyor,obek_5,82.176053
2,Erkek,18-30,Bekar,Lise Mezunu,Düzenli ve Ücretli Bir İşi Var,268582.732360,Küçük Şehir,Giyim,9924.379414,55.353236,Ediyor,obek_8,141.661426
3,Erkek,51-60,Evli,Lise Mezunu,Kendi İşinin Sahibi,327721.837865,Kırsal,Giyim,6417.780714,15.417239,Etmiyor,obek_3,17.097397
4,Erkek,31-40,Evli,Ortaokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,397431.626948,Büyük Şehir,Giyim,7886.663061,14.689837,Etmiyor,obek_3,20.470459
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,Kadın,18-30,Bekar,Yüksekokul Mezunu,Kendi İşinin Sahibi,259190.078834,Küçük Şehir,Ev ve Mobilya,9559.156534,41.883661,Ediyor,obek_8,117.627785
2336,Kadın,31-40,Evli,Lise Mezunu,İşsiz veya Düzenli Bir İşi Yok,194580.563177,Kırsal,Ev ve Mobilya,4070.232154,20.082996,Etmiyor,obek_2,45.659622
2337,Erkek,>60,Evli,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,357730.521969,Büyük Şehir,Ev ve Mobilya,22589.204381,18.922305,Etmiyor,obek_7,100.691379
2338,Erkek,18-30,Evli,Doktora Ötesi,Düzenli ve Ücretli Bir İşi Var,213179.637202,Küçük Şehir,Spor Malzemeleri,4205.390562,10.758108,Etmiyor,obek_5,97.778913


In [326]:
"""plt.figure(figsize=(30, 30))
sns.heatmap(real_extended.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Korelasyon Matrisi Heatmap')
plt.show()"""

"plt.figure(figsize=(30, 30))\nsns.heatmap(real_extended.corr(), annot=True, cmap='coolwarm', center=0)\nplt.title('Korelasyon Matrisi Heatmap')\nplt.show()"

In [327]:
real_extended['Maddi Oran'] = real_extended['Yıllık Ortalama Gelir'].values / \
    real_extended['Yıllık Ortalama Satın Alım Miktarı'].values
real_extended['Maddi Oran'].fillna(0, inplace=True)

In [328]:
test_df['Maddi Oran'] = test_df['Yıllık Ortalama Gelir'] / \
    test_df['Yıllık Ortalama Satın Alım Miktarı']
test_df['Maddi Oran'].fillna(0, inplace=True)

In [329]:
real_extended['Alim Orani'] = real_extended['Yıllık Ortalama Sipariş Verilen Ürün Adedi'] / \
    real_extended['Yıllık Ortalama Sepete Atılan Ürün Adedi'].values
real_extended['Alim Orani'].fillna(0, inplace=True)

In [330]:
test_df['Alim Orani'] = (test_df['Yıllık Ortalama Sipariş Verilen Ürün Adedi'] /
                         test_df['Yıllık Ortalama Sepete Atılan Ürün Adedi']).values
test_df['Alim Orani'].fillna(0, inplace=True)

In [331]:
label_encoders = {}
for column in categorical_features:
    le = LabelEncoder()
    real_extended[column] = le.fit_transform(real_extended[column])
    label_encoders[column] = le

real_extended

,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Maddi Oran,Alim Orani
0,1,1,0,5,0,748266.440879,0,0,32010.900881,37.310644,1,obek_4,102.936904,23.375363,0.362461
1,0,4,1,3,2,246298.609605,2,4,4145.729891,11.554851,1,obek_5,82.176053,59.410192,0.140611
2,0,0,0,3,0,268582.732360,2,2,9924.379414,55.353236,0,obek_8,141.661426,27.062925,0.390743
3,0,3,1,3,2,327721.837865,3,2,6417.780714,15.417239,1,obek_3,17.097397,51.064667,0.901730
4,0,1,1,4,3,397431.626948,0,2,7886.663061,14.689837,1,obek_3,20.470459,50.392875,0.717612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,1,0,0,6,2,259190.078834,2,1,9559.156534,41.883661,0,obek_8,117.627785,27.114325,0.356069
2336,1,1,1,3,3,194580.563177,3,1,4070.232154,20.082996,1,obek_2,45.659622,47.805765,0.439841
2337,0,4,1,5,0,357730.521969,0,1,22589.204381,18.922305,1,obek_7,100.691379,15.836349,0.187924
2338,0,0,1,1,0,213179.637202,2,4,4205.390562,10.758108,1,obek_5,97.778913,50.691995,0.110025


In [332]:
label_encoders = {}
for column in categorical_features:
    le = LabelEncoder()
    test_df[column] = le.fit_transform(test_df[column])
    label_encoders[column] = le

test_df

,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Yıllık Ortalama Sepete Atılan Ürün Adedi,Maddi Oran,Alim Orani
0,0,3,1,5,0,365331.930346,1,2,5566.587664,15.442635,1,19.646460,65.629422,0.786026
1,0,2,0,3,3,414899.236174,0,4,6756.766203,14.957246,1,19.675863,61.405001,0.760182
2,1,4,1,2,3,152825.872271,2,2,4493.915682,10.907930,1,46.904518,34.007285,0.232556
3,1,3,1,1,0,489052.908472,1,3,42670.646595,32.333021,1,48.083609,11.461108,0.672433
4,0,0,1,5,2,323635.711534,2,2,5959.266948,4.519968,1,12.752491,54.307974,0.354438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,1,0,0,6,2,259190.078834,2,1,9559.156534,41.883661,0,117.627785,27.114325,0.356069
2336,1,1,1,3,3,194580.563177,3,1,4070.232154,20.082996,1,45.659622,47.805765,0.439841
2337,0,4,1,5,0,357730.521969,0,1,22589.204381,18.922305,1,100.691379,15.836349,0.187924
2338,0,0,1,1,0,213179.637202,2,4,4205.390562,10.758108,1,97.778913,50.691995,0.110025


In [333]:
target_encoder = LabelEncoder()
real_extended['Öbek İsmi'] = target_encoder.fit_transform(
    real_extended['Öbek İsmi'])

real_extended

,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Maddi Oran,Alim Orani
0,1,1,0,5,0,748266.440879,0,0,32010.900881,37.310644,1,3,102.936904,23.375363,0.362461
1,0,4,1,3,2,246298.609605,2,4,4145.729891,11.554851,1,4,82.176053,59.410192,0.140611
2,0,0,0,3,0,268582.732360,2,2,9924.379414,55.353236,0,7,141.661426,27.062925,0.390743
3,0,3,1,3,2,327721.837865,3,2,6417.780714,15.417239,1,2,17.097397,51.064667,0.901730
4,0,1,1,4,3,397431.626948,0,2,7886.663061,14.689837,1,2,20.470459,50.392875,0.717612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,1,0,0,6,2,259190.078834,2,1,9559.156534,41.883661,0,7,117.627785,27.114325,0.356069
2336,1,1,1,3,3,194580.563177,3,1,4070.232154,20.082996,1,1,45.659622,47.805765,0.439841
2337,0,4,1,5,0,357730.521969,0,1,22589.204381,18.922305,1,6,100.691379,15.836349,0.187924
2338,0,0,1,1,0,213179.637202,2,4,4205.390562,10.758108,1,4,97.778913,50.691995,0.110025


In [334]:
X = real_extended.drop('Öbek İsmi', axis=1)
y = real_extended['Öbek İsmi']

# Scaling the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [335]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.33, random_state=2)
print(X_train.shape)
print("*"*10)
print(X_test.shape)
print("*"*10)
print(y_train)
print("*"*10)
print(y_test.shape)

(5226, 14)
**********
(2574, 14)
**********
815     0
59      6
159     3
280     3
3372    2
       ..
3606    4
244     2
1177    1
2575    6
1876    5
Name: Öbek İsmi, Length: 5226, dtype: int32
**********
(2574,)


In [336]:
X

,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Yıllık Ortalama Sepete Atılan Ürün Adedi,Maddi Oran,Alim Orani
0,1,1,0,5,0,748266.440879,0,0,32010.900881,37.310644,1,102.936904,23.375363,0.362461
1,0,4,1,3,2,246298.609605,2,4,4145.729891,11.554851,1,82.176053,59.410192,0.140611
2,0,0,0,3,0,268582.732360,2,2,9924.379414,55.353236,0,141.661426,27.062925,0.390743
3,0,3,1,3,2,327721.837865,3,2,6417.780714,15.417239,1,17.097397,51.064667,0.901730
4,0,1,1,4,3,397431.626948,0,2,7886.663061,14.689837,1,20.470459,50.392875,0.717612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,1,0,0,6,2,259190.078834,2,1,9559.156534,41.883661,0,117.627785,27.114325,0.356069
2336,1,1,1,3,3,194580.563177,3,1,4070.232154,20.082996,1,45.659622,47.805765,0.439841
2337,0,4,1,5,0,357730.521969,0,1,22589.204381,18.922305,1,100.691379,15.836349,0.187924
2338,0,0,1,1,0,213179.637202,2,4,4205.390562,10.758108,1,97.778913,50.691995,0.110025


In [344]:
test_data_scaled = scaler.transform(test_df)

(2340, 14)

In [338]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the RandomForest Classifier
clf = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)

# Train the classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(
    y_test, y_pred, target_names=target_encoder.classes_)

accuracy

0.9708624708624709

In [345]:
test_predictions = clf.predict(test_data_scaled)
test_predictions_labels = target_encoder.inverse_transform(test_predictions)

['obek_3' 'obek_3' 'obek_2' ... 'obek_7' 'obek_5' 'obek_6']


In [340]:
test_predictions

array([2, 2, 1, ..., 6, 4, 5])

In [341]:
submission_df = pd.DataFrame(
    {'id': submission_ids, 'Öbek İsmi': test_predictions_labels})
submission_df.to_csv('submission_EXTENDED.csv', index=False)